In [2]:
from langgraph.graph import StateGraph, END, START
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv('credintials.env')

True

In [3]:
# setting up the LLM

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

In [11]:
from pydantic import BaseModel, Field
from typing import Literal, List

class OrchestratorFormater(BaseModel):
    city_name: str = Field(description="The name of the city to get the weather for.")

In [15]:
import requests

def get_location_key(city_name, api_key):
    """Convert city name to AccuWeather location key"""
    url = "http://dataservice.accuweather.com/locations/v1/cities/search"
    params = {
        "apikey": api_key,
        "q": city_name
    }
    
    response = requests.get(url, params=params)
    data = response.json()

    # print(data)  # See what you're getting


    if data:
        return data[0]["Key"]  # Returns location key
    return None

In [16]:
weather_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a weather agent that provides weather information based on user queries, you should extract the city name from the user query."),
    ("user", "{query}")
])

In [17]:
weather_pipe = weather_prompt | llm.with_structured_output(OrchestratorFormater)